In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as partial
%matplotlib inline

In [2]:
# 导入训练数据和测试数据
df_train = pd.read_csv("train_data_public.csv")
df_test = pd.read_csv("test_public.csv")
df_train = df_train.drop(['id'],axis=1)
df_test = df_test.drop(['id'],axis=1)
df_train.head()

,text,BIO_anno,class
0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1


In [44]:
# 加载TokenEmbedding
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding, 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

# 查看token_embedding详情
print(token_embedding)

[2022-08-25 22:10:28,707] [    INFO] - Loading token embedding...
[2022-08-25 22:11:03,845] [    INFO] - Finish loading embedding vector.
[2022-08-25 22:11:03,889] [    INFO] - Token Embedding info:             
Unknown index: 635963             
Unknown token: [UNK]             
Padding index: 635964             
Padding token: [PAD]             
Shape :[635965, 300]


Object   type: TokenEmbedding(635965, 300, padding_idx=635964, sparse=False)             
Unknown index: 635963             
Unknown token: [UNK]             
Padding index: 635964             
Padding token: [PAD]             
Parameter containing:
Tensor(shape=[635965, 300], dtype=float32, place=Place(cpu), stop_gradient=False,
       [[-0.24200200,  0.13931701,  0.07378800, ...,  0.14103900,
          0.05592300, -0.08004800],
        [-0.08671700,  0.07770800,  0.09515300, ...,  0.11196400,
          0.03082200, -0.12893000],
        [-0.11436500,  0.12201900,  0.02833000, ...,  0.11068700,
          0.03607300, -0.13763499],
        ...,
        [ 0.02628800, -0.00008300, -0.00393500, ...,  0.00654000,
          0.00024600, -0.00662600],
        [-0.02472960, -0.01754892,  0.02292210, ..., -0.02756814,
         -0.02185878, -0.02111119],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])


In [169]:
import jieba
train_vector = []
mark = 0
for i in df_train['text']:
    # 利用jieba进行分词
    temp = jieba.lcut(i)
    vec_temp = []
    # word2vec
    for j in temp:
        vec_temp.append(token_embedding.search(j))
    train_vector.append(vec_temp)
    mark += 1
    print("\r完成%.2f %%" % (mark *100 / len(df_train)), end="")

完成100.00 %

In [270]:
# 利用累加把词向量合成为句向量
mark = 0
tain_sentence = []
for i in train_vector:
    vec_temp = np.zeros(300)
    for j in i:
        vec_temp = np.add(vec_temp,np.array(j))
    vec_temp = np.divide(vec_temp,len(i))
    tain_sentence.append(vec_temp)
    mark += 1
    print("\r完成%.2f %%" % (mark *100 / len(df_train)), end="")
train_temp = np.reshape(tain_sentence,(len(tain_sentence),300))

完成100.00 %

In [272]:
# 修改标签值令中立为0.5
label_trans = np.array(df_train['class'])
label_trans = np.where(label_trans==2,0.5,label_trans)

# 切分训练集和验证集
import paddle
ratio = 0.8
offset = int(ratio*len(df_train))
x_train = paddle.to_tensor(train_temp[:offset])
y_train = paddle.to_tensor(label_trans[:offset])
x_dev = paddle.to_tensor(train_temp[offset:])
y_dev = paddle.to_tensor(label_trans[offset:])

In [275]:
class MyDataset(paddle.io.Dataset):
    """
    步骤一:继承paddle.io.Dataset类
    """
    def __init__(self,feature,label):
        """
        步骤二:实现构造函数,定义数据读取方式,划分训练和测试数据集
        """
        super(MyDataset, self).__init__()
        self.features = paddle.to_tensor(feature, dtype='float32')
        self.labels = paddle.to_tensor(label, dtype='float32')

    def __getitem__(self, index):
        """
        步骤三:实现__getitem__方法,定义指定index时如何获取数据,并返回单条数据（训练数据）
        """
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        """
        步骤四:实现__len__方法,返回数据集总数目
        """
        return len(self.labels)

train_dataset = MyDataset(x_train,y_train)

In [284]:
# 情感分类模型
import paddle.nn.layer
class myNet(paddle.nn.Layer):
    def __init__(self):
        super(myNet, self).__init__()
        self.l1 = paddle.nn.Linear(300,900,)
        self.l2 = paddle.nn.Linear(900,450,)
        self.l3 = paddle.nn.Linear(450,225,)
        self.l4 = paddle.nn.Linear(225,15,)
        self.l5 = paddle.nn.Linear(15,1,)
        self.softmax = paddle.nn.layer.Softmax()
        self.sigmoid = paddle.nn.layer.Sigmoid()
        self.dropout = paddle.nn.layer.Dropout(0.3)

    def forward(self,pred):
        pred = self.l1(pred)
        pred = self.dropout(pred)
        pred = self.sigmoid(pred)
        pred = self.l2(pred)
        pred = self.dropout(pred)
        pred = self.sigmoid(pred)
        pred = self.l3(pred)
        pred = self.sigmoid(pred)
        pred = self.l4(pred)
        pred = self.sigmoid(pred)
        pred = self.l5(pred)

In [277]:
from paddle.metric import Accuracy

model = paddle.Model(myNet())
# 配置模型
model.prepare(
    paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),
    paddle.nn.L1Loss(),
    Accuracy()
    )

In [287]:
# 训练模型
model.fit(train_dataset,
        epochs=2,
        batch_size=64,
        verbose=1
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/2


TypeError: unsupported operand type(s) for +: 'NoneType' and 'list'

In [260]:
model.forward(paddle.to_tensor(train_temp[0]))
# train_temp[0]